In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

chrome_driver_path = 'chromedriver/chromedriver.exe'
chrome_options = Options()

chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

csv_filename = "dorahacks.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["title", "details"])

    try:
        url = 'https://dorahacks.io/hackathon'
        driver.get(url)
        time.sleep(5)

        hackathon_xpath_pattern = "//*[@id='hackathon-list-page']/div[3]/div/div/ul/li/a"

        loaded_hackathons = 0

        while True:
            time.sleep(2)  # 페이지 로드 대기
            hackathon_elements = driver.find_elements(By.XPATH, hackathon_xpath_pattern)
            total_hackathons = len(hackathon_elements)
            print(f'현재 로드된 해커톤 개수: {total_hackathons}')
            
            for i in range(loaded_hackathons, total_hackathons):
                try:
                    hackathon_elements = driver.find_elements(By.XPATH, hackathon_xpath_pattern)  # 최신 요소 리스트 가져오기
                    hackathon_element = hackathon_elements[i]
                    hackathon_url = hackathon_element.get_attribute('href')
                    
                    split_url = hackathon_url.split('/')
                    if len(split_url) > 4 and split_url[-1] != 'detail':
                        hackathon_url = '/'.join(split_url[:5]) + '/detail'
                    
                    # 새 탭에서 해커톤 페이지 열기
                    driver.execute_script("window.open(arguments[0]);", hackathon_url)
                    driver.switch_to.window(driver.window_handles[1])  # 새 탭으로 이동
                    time.sleep(3)
                    
                    try:
                        title_element = driver.find_element(By.XPATH, "//*[@id='hackathon-detail']/div/div[1]/div/div/div[4]/h1")
                        title = title_element.text
                    except:
                        print(f'[{hackathon_url}] 제목을 찾을 수 없음. 다음으로 이동.')
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        continue
                    
                    try:
                        content_element = driver.find_element(By.XPATH, "//*[@id='hackathon-detail']/div/div[3]/div[2]/div/div/div/div/div/div[2]")
                        content = content_element.text
                    except:
                        print(f'[{hackathon_url}] 상세 내용을 찾을 수 없음.')
                        content = "정보 없음"
                    
                    writer.writerow([title, content])
                    print(f'저장됨: {title}')
                    
                    driver.close()  # 새 탭 닫기
                    driver.switch_to.window(driver.window_handles[0])  # 원래 탭으로 복귀
                    time.sleep(3)
                    
                except Exception as e:
                    print(f'오류 발생: {e}')
            
            loaded_hackathons = total_hackathons
            
            try:
                view_more_button = driver.find_element(By.XPATH, "//*[@id='hackathon-list-page']/div[3]/div/div/div")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                driver.execute_script("arguments[0].scrollIntoView();", view_more_button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", view_more_button)
                time.sleep(5)
            except:
                print('View More 버튼을 찾을 수 없음 또는 클릭 실패: 더 이상 데이터 없음')
                break 

    finally:
        driver.quit()

print(f'크롤링 완료! 데이터가 {csv_filename}에 저장되었습니다.')

현재 로드된 해커톤 개수: 12
저장됨: Sonic DeFAI Hackathon
저장됨: Seeds Of Agentic Future
저장됨: Injective AI Agent Hackathon
저장됨: HackSecret 5
저장됨: AI MegaWave Hackathon
저장됨: Core Global Gaming Hackathon
저장됨: HackTheBurgh XI
저장됨: Vyper/Polygon AI Agent Workshop
저장됨: (IRL) Flare x Google Verifiable AI Hackathon
저장됨: (Virtual) Flare x Google Verifiable AI Hackathon
저장됨: Build & Brew - Espresso Global Hackathon
저장됨: BUIDL BATTLE | The Bitcoin Builders Tournament
현재 로드된 해커톤 개수: 24
저장됨: Verifiable AI, save the world with cheqd
저장됨: Cambrian Hack
저장됨: Ancient8 OP Grant Program
저장됨: DeSci Mania at Zo
저장됨: Polkadot Blockspace Hackerhouse - Bounty Board - Hackathon
저장됨: Hack The Grid — Level 2
저장됨: NERD Network: ETHDenver
저장됨: Polkadot Philippines Hackathon 2025
저장됨: TON Global BTCFi Hackathon
저장됨: Naija HackAtom
저장됨: Pears Hackathon
저장됨: BananaHack At Zo
현재 로드된 해커톤 개수: 36
저장됨: Avalanche | NeuraNode Game Jam
저장됨: The Gated NFTs World Cup
저장됨: ETHBucharest 2025
저장됨: Hack To The HIVE
저장됨: SynapHack 2.0
저장됨: AI & 

In [2]:
import csv
import re
from langdetect import detect, DetectorFactory, LangDetectException

DetectorFactory.seed = 0  # 언어 감지 결과의 일관성을 위해 시드 설정

def filter_csv(input_csv, output_csv):
    with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # 헤더 유지
        header = next(reader)
        writer.writerow(header)
        
        for row in reader:
            title, content = row
            
            # 빈 값 또는 너무 짧은 텍스트 제외
            if not title.strip() or not content.strip() or len(title) < 3 or len(content) < 3:
                print(f'제거됨: {title} (내용이 너무 짧거나 없음)')
                continue
            
            # 제목과 내용에서 중국어 포함 여부 확인
            if re.search(r'[\u4e00-\u9fff]', title) or re.search(r'[\u4e00-\u9fff]', content):
                print(f'제거됨: {title} (중국어 포함)')
                continue  # 중국어 포함된 데이터는 저장 안 함
            
            # 영어가 아닌 경우 제거 (오류 방지 처리 추가)
            try:
                if detect(title) != 'en' or detect(content) != 'en':
                    print(f'제거됨: {title} (영어가 아님)')
                    continue
            except LangDetectException:
                print(f'제거됨: {title} (언어 감지 실패)')
                continue
            
            writer.writerow([title, content])
    
    print(f'필터링 완료! 결과가 {output_csv}에 저장되었습니다.')

# 기존 크롤링된 CSV에서 필터링 실행
filter_csv(csv_filename, csv_filename.replace('.csv', '_filtered.csv'))

제거됨: Ancient8 OP Grant Program (영어가 아님)
제거됨: DeSci Mania at Zo (영어가 아님)
제거됨: NERD Network: ETHDenver (영어가 아님)
제거됨: Polkadot Philippines Hackathon 2025 (영어가 아님)
제거됨: Naija HackAtom (영어가 아님)
제거됨: BananaHack At Zo (영어가 아님)
제거됨: Avalanche | NeuraNode Game Jam (영어가 아님)
제거됨: ETHBucharest 2025 (영어가 아님)
제거됨: SynapHack 2.0 (영어가 아님)
제거됨: CodeForge'25 (영어가 아님)
제거됨: Solaris (영어가 아님)
제거됨: ETH Oxford 2025 (영어가 아님)
제거됨: ElleHacks 2025 (영어가 아님)
제거됨: MIT iQuHACK 2025 Builders Grant (영어가 아님)
제거됨: ETHKyiv Impulse: AI Agents Reloaded (영어가 아님)
제거됨: HackRU Spring 2025 (영어가 아님)
제거됨: Hackathon@WEF 2025 (언어 감지 실패)
제거됨: uOttaHack 7 (영어가 아님)
제거됨: HackWiE 3.O (영어가 아님)
제거됨: Watson Code Fest 2024 (영어가 아님)
제거됨: Aleph de Verano Hackathon (영어가 아님)
제거됨: Polkadot Vietnam Hackathon 2025 (영어가 아님)
제거됨: LNMHacks 7.0 (영어가 아님)
제거됨: XION BELIEVATHON (영어가 아님)
제거됨: LXDAO Casual Hackathon：AI 开发工具 (중국어 포함)
제거됨: Ancient8 Builder Jam (영어가 아님)
제거됨: ETHKyiv Impulse: Based Edition (영어가 아님)
제거됨: HIZE HACKATHON 2025 (영어가 아님)
제거됨: HackEli